In [11]:
# !wget --no-check-certificate https://huggingface.co/datasets/mesolitica/pseudolabel-malaysian-youtube-whisper-large-v3/resolve/main/pseudolabel.jsonl

In [22]:
import string

punct = set(string.punctuation)

def remove_punct(s):
    return ''.join([c for c in s if c not in punct])
    
selected = [
    'terima kasih kerana menonton',
    'terima kasih',
]

In [30]:
import json
from tqdm import tqdm
from sklearn.feature_extraction.text import CountVectorizer

with open('semisupervised-whisper-large-v3.jsonl', 'w') as fopen_l:
    with open('pseudolabel.jsonl') as fopen:
        for l in tqdm(fopen):
            l = json.loads(l)
            ms = l['predict_ms'].split('<|transcribe|>')[1].strip()[:-13]
            en = l['predict_en'].split('<|transcribe|>')[1].strip()[:-13]

            ms_lower = ms.lower()

            if any([s in ms_lower for s in selected]):
                continue

            if len(ms) < 10 or len(en) < 10:
                continue

            splitted = ms.split()
            ratio = (len(set(splitted)) / len(splitted))
            if ratio < 0.5:
                continue

            splitted = en.split()
            ratio = (len(set(splitted)) / len(splitted))
            if ratio < 0.5:
                continue
    
            text = f'below is the text from pseudolabel youtube videos in Malay and English,\n\nMalay:\n{ms}\n\nEnglish:\n{en}'
            fopen_l.write(f'{json.dumps(text)}\n')
            fopen_l.flush()

2221856it [01:46, 20927.98it/s]


In [31]:
!head -n 10 semisupervised-whisper-large-v3.jsonl

"below is the text from pseudolabel youtube videos in Malay and English,\n\nMalay:\nanda tahu keuntungan boleh lebih tinggi daripada keuntungan kewangan rumah maka saya tidak akan mencari dalam akaun saya akan mencari ke dalam ethereum atau beberapa crypto punks bergantung pada faktor risiko anda kerana rumah kajang dihantar tidak mengganggu dsr saya sejauh ini jadi sekarang apa posisi saya untuk mendapatkan kewangan ketiga jadi mungkin setelah melihat sekeliling saya menemui seorang penjual yang dapat menutupi perhubungan tetapi bank hanya menerima 70% dari itu saya boleh membayar perbezaan dengan menggunakan wang ini kerana sekali lagi ia menyusahkan saya dan aset tetapi jika anda tidak selesa dengan mencari\n\nEnglish:\nyou know the returns can be higher than the savings of the housing loan interest then i will not put in the account i'll put into ethereum or some crypto punks depending on your risk factor then because of the kajang house being let out it does not affect my dsr as m

In [32]:
from huggingface_hub import HfApi
api = HfApi()
api.upload_file(
    path_or_fileobj="semisupervised-whisper-large-v3.jsonl",
    path_in_repo="semisupervised-whisper-large-v3.jsonl",
    repo_id="malaysia-ai/dedup-text-dataset",
    repo_type="dataset",
)


semisupervised-whisper-large-v3.jsonl:   0%|          | 0.00/1.43G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/malaysia-ai/dedup-text-dataset/commit/2a267ce703170cfce670231b11b9f187530f8bd2', commit_message='Upload semisupervised-whisper-large-v3.jsonl with huggingface_hub', commit_description='', oid='2a267ce703170cfce670231b11b9f187530f8bd2', pr_url=None, pr_revision=None, pr_num=None)